# SQLite example

This example showcases hooking up an LLM to answer questions over a database.

This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

# start ss

In [2]:
import pandas as pd
import sqlite3

In [3]:
sqldb_str = "/Users/scottschmidt/sqlitedbs/tkc.db"

In [ ]:
con = sqlite3.connect(sqldb_str)

In [ ]:
df_mthstatus = pd.read_csv('../../../../ssData/tkcmonthlystatus.csv')
df_manpower = pd.read_csv('../../../../ssData/tkcgmp.csv')

In [ ]:
[df_mthstatus.shape, df_manpower.shape]

In [ ]:
def remove_last_space(x):
    return x[:-1] if x[-1] == ' ' else x

def remove_last_dash(x):
    return x[:-1] if x[-1] == '-' else x 

In [ ]:
df_manpower['contract'] = df_manpower['project_number'].apply(remove_last_space)

In [ ]:
df_mthstatus['contract'] = df_mthstatus['contract'].apply(remove_last_dash)

In [ ]:
df_mthstatus.to_sql("mthstatus", con, if_exists="replace")

In [ ]:
df_manpower.to_sql("manpower", con, if_exists="replace")

In [ ]:
cursor = con.cursor()

In [ ]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

In [ ]:
cursor.execute(sql_query)
print(cursor.fetchall())

In [ ]:
con.execute("DROP TABLE df_mp")

In [ ]:
cursor.execute("Select * from manpower limit 10")
print(cursor.fetchall())

In [ ]:
cursor.execute("Select * from mthstatus limit 10")
print(cursor.fetchall())

In [ ]:
for row in con.execute("pragma table_info('manpower')").fetchall():
    print(row)

In [ ]:
for row in con.execute("pragma table_info('mthstatus')").fetchall():
    print(row)

In [ ]:
con.close()

In [4]:
sqldb_str = "sqlite:////Users/scottschmidt/sqlitedbs/tkc.db"
db = SQLDatabase.from_uri(sqldb_str)
llm = OpenAI(temperature=0)

In [5]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [9]:
sa = db_chain.run("How many unique contracts are there in table mthstatus?")



> Entering new SQLDatabaseChain chain...
How many unique contracts are there in table mthstatus? 
SQLQuery: SELECT COUNT(DISTINCT contract) FROM mthstatus;
SQLResult: [(1045,)]
Answer: There are 1045 unique contracts in table mthstatus.
> Finished chain.


In [10]:
sa = db_chain.run("List the tables in the database?")



> Entering new SQLDatabaseChain chain...
List the tables in the database? 
SQLQuery: SELECT name FROM sqlite_master WHERE type='table';
SQLResult: [('mthstatus',), ('manpower',)]
Answer: The tables in the database are mthstatus and manpower.
> Finished chain.


In [11]:
sa = db_chain.run("List the columns in the manpower table")



> Entering new SQLDatabaseChain chain...
List the columns in the manpower table 
SQLQuery: SELECT * FROM 'manpower' LIMIT 1;
SQLResult: [(0, 1656486, '21221', '2021-09-01', 22120632.0, '"Central Ceiling North Inc."', 235507.0, '"Carpentry"', 16)]
Answer: The columns in the manpower table are index, projectid, project_number, mthyr, vendorid, vendorname, tradeid, tradename, and man_hours.
> Finished chain.


In [19]:
sa = db_chain.run("How many rows are there in table mthstatus?")



> Entering new SQLDatabaseChain chain...
How many rows are there in table mthstatus? 
SQLQuery: SELECT COUNT(*) FROM mthstatus;
SQLResult: [(31112,)]
Answer: There are 31112 rows in table mthstatus.
> Finished chain.


In [ ]:
sa = db_chain.run("subtract the sum of projprofit in mth 2022-12-01 from the sum of projprofit in mth 2023-01-01")

In [22]:
sa = db_chain.run("what is the sum of projprofit in mth 2023-03-01?")



> Entering new SQLDatabaseChain chain...
what is the sum of projprofit in mth 2023-03-01? 
SQLQuery: SELECT SUM(projprofit) FROM mthstatus WHERE mth = '2023-03-01';
SQLResult: [(99859406.02999997,)]
Answer: The sum of projprofit in mth 2023-03-01 is 99859406.03.
> Finished chain.


In [23]:
sa = db_chain.run("what is the sum of projprofit in mth 2023-02-01?")



> Entering new SQLDatabaseChain chain...
what is the sum of projprofit in mth 2023-02-01? 
SQLQuery: SELECT SUM(projprofit) FROM mthstatus WHERE mth = '2023-02-01';
SQLResult: [(99388540.85,)]
Answer: The sum of projprofit in mth 2023-02-01 is 99388540.85.
> Finished chain.


In [24]:
sa = db_chain.run("subtract the sum of projprofit in mth 2023-02-01 from the sum of projprofit in mth 2023-03-01")



> Entering new SQLDatabaseChain chain...
subtract the sum of projprofit in mth 2023-02-01 from the sum of projprofit in mth 2023-03-01 
SQLQuery: SELECT SUM(projprofit) FROM mthstatus WHERE mth = '2023-02-01' OR mth = '2023-03-01'
SQLResult: [(199247946.88000005,)]
Answer: 199247946.88000005
> Finished chain.


In [ ]:
type(sa)

# End ss

In [ ]:
#ss
# Valid SQLite URL forms are:
#  sqlite:///:memory: (or, sqlite://)
#  sqlite:///relative/path/to/file.db
#  sqlite:////absolute/path/to/file.db
db = SQLDatabase.from_uri("sqlite:////Users/scottschmidt/sqlitedbs/Chinook.db")

In [ ]:
sqldb_str = "sqlite:////Users/scottschmidt/sqlitedbs/Chinook.db"
db = SQLDatabase.from_uri(sqldb_str)
llm = OpenAI(temperature=0)

**NOTE:** For data-sensitive projects, you can specify `return_direct=True` in the `SQLDatabaseChain` initialization to directly return the output of the SQL query without any additional formatting. This prevents the LLM from seeing any contents within the database. Note, however, the LLM still has access to the database scheme (i.e. dialect, table and key names) by default.

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True, return_intermediate_steps=True)

In [ ]:
sa = db_chain.run("How many employees are there?")

## Customize Prompt
You can also customize the prompt that is used. Here is an example prompting it to understand that foobar is the same as the Employee table

In [ ]:
sa

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True)

In [ ]:
db_chain.run("How many employees are there in the foobar table?")

## Return Intermediate Steps

You can also return the intermediate steps of the SQLDatabaseChain. This allows you to access the SQL statement that was generated, as well as the result of running that against the SQL Database.

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True, return_intermediate_steps=True)

In [ ]:
result = db_chain("How many employees are there in the foobar table?")
result["intermediate_steps"]

In [ ]:
result["intermediate_steps"]

## Choosing how to limit the number of rows returned
If you are querying for several rows of a table you can select the maximum number of results you want to get by using the 'top_k' parameter (default is 10). This is useful for avoiding query results that exceed the prompt max length or consume tokens unnecessarily.

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, top_k=3)

In [ ]:
db_chain.run("What are some example tracks by composer Johann Sebastian Bach?")

## Adding example rows from each table
Sometimes, the format of the data is not obvious and it is optimal to include a sample of rows from the tables in the prompt to allow the LLM to understand the data before providing a final query. Here we will use this feature to let the LLM know that artists are saved with their full names by providing two rows from the `Track` table.

In [ ]:
db = SQLDatabase.from_uri(
    "sqlite:///../../../../notebooks/Chinook.db",
    include_tables=['Track'], # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2)

The sample rows are added to the prompt after each corresponding table's column information:

In [ ]:
print(db.table_info)

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
db_chain.run("What are some example tracks by Bach?")

### Custom Table Info
In some cases, it can be useful to provide custom table information instead of using the automatically generated table definitions and the first `sample_rows_in_table_info` sample rows. For example, if you know that the first few rows of a table are uninformative, it could help to manually provide example rows that are more diverse or provide more information to the model. It is also possible to limit the columns that will be visible to the model if there are unnecessary columns. 

This information can be provided as a dictionary with table names as the keys and table information as the values. For example, let's provide a custom definition and sample rows for the Track table with only a few columns:

In [ ]:
custom_table_info = {
    "Track": """CREATE TABLE Track (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL,
	"Composer" NVARCHAR(220),
	PRIMARY KEY ("TrackId")
)

SELECT * FROM 'Track' LIMIT 3;
TrackId	Name	Composer
1	For Those About To Rock (We Salute You)	Angus Young, Malcolm Young, Brian Johnson
2	Balls to the Wall	None
3	My favorite song ever	The coolest composer of all time"""
}

In [ ]:
db = SQLDatabase.from_uri(
    "sqlite:///../../../../notebooks/Chinook.db",
    include_tables=['Track', 'Playlist'],
    sample_rows_in_table_info=2,
    custom_table_info=custom_table_info)

print(db.table_info)

Note how our custom table definition and sample rows for `Track` overrides the `sample_rows_in_table_info` parameter. Tables that are not overriden by `custom_table_info`, in this example `Playlist`, will have their table info gathered automatically as usual.

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
db_chain.run("What are some example tracks by Bach?")

## SQLDatabaseSequentialChain

Chain for querying SQL database that is a sequential chain.

The chain is as follows:

    1. Based on the query, determine which tables to use.
    2. Based on those tables, call the normal SQL database chain.

This is useful in cases where the number of tables in the database is large.

In [ ]:
from langchain.chains import SQLDatabaseSequentialChain
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")

In [ ]:
chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [ ]:
chain.run("How many employees are also customers?")

# pandas 

In [ ]:
df_mthstatus.contract.apply(lambda x: len(x)).unique()

In [ ]:
df_manpower.project_number.apply(lambda x: len(x)).unique()

In [ ]:

df_manpower[df_manpower.project_number.apply(lambda x: len(x)==6)].project_number

In [ ]:
df_manpower.contract.apply(lambda x: len(x)).unique()

In [ ]:
df_mthstatus.contract.apply(lambda x: len(x)).unique()